# Recurrent Residual Convolutional Neural Network

### Simon Laurent Lebailly, 2549365, s9sileba@teams.uni-saarland.de
### Christian Mathieu Schmidt, 2537621, s9cmscmi@teams.uni-saarland.de

## Import libaries

In [ ]:
import torch
import torchvision
import numpy as np

## Define Hyperparameters

In [ ]:
batch_size = 4
learning_rate = 0.0001

## Import dataset Cityscapes

In [ ]:
from torch import utils
from torchvision import transforms
from torchvision import datasets

#Define transformation for dataset
transform = transforms.RandomCrop(...)

root_path = 'data'

#Import dataset for training
train_set = datasets.Cityscapes(root_path, transform=transform)
quantity_train = len(train_set)
print('Quantity training data: '+ str(quantity_train))

train_loader = utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)


#Import dataset for training
train_set = datasets.Cityscapes(root_path, transform=transform)
quantity_train = len(train_set)
print('Quantity training data: '+ str(quantity_train))



#Import dataset for training
train_set = datasets.Cityscapes(root_path, transform=transform)
quantity_train = len(train_set)
print('Quantity training data: '+ str(quantity_train))

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)